In [4]:
import smtplib

from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from email.mime.text import MIMEText

import getpass
sender = 'ypjun@yonsei.ac.kr'
#password = getpass.getpass("login: ypjun@yonsei.ac.kr\nPassword: ")
password = 'ttcdkghgzclncuai'

SUCCESS = 0
FILEERROR = 1
SMTPERROR = 2
def sendMail(receiver, title, body, attached):
    message = MIMEMultipart('alternative')
    message['Subject'] = title
    message['From'] = "Young Pyo Jun" # sender
    message['To'] = receiver
    content = MIMEText(body)
    message.attach(content)
    # File attach
    fname = attached
    with open(fname, "rb") as f:
        if not f: return FILEERROR
        part = MIMEApplication(f.read(), name=attached)
        message.attach(part)

    with smtplib.SMTP_SSL('smtp.gmail.com', 465) as server:
#    with smtplib.SMTP_SSL('zmsmtp.mailplug.com', 465) as server:
        if not server: return SMTPERROR
        server.ehlo()
        server.login(sender, password)
        server.send_message(message)
    return SUCCESS

#sendMail('ypjun7@gmail.com', "Test mail", "Hello~~", "Scoring_강현웅_2021240476.txt")

import pandas as pd
import os
import tkinter as tk
from tkinter import filedialog

def SelectDir():
    fname = ""
    window = tk.Tk()
    window.file = filedialog.askdirectory(initialdir=".", title="Select")
    if window.file != None:
        fname = window.file
    window.destroy()
    return fname

def getHead(fname):
    stHead = ''
    with open(fname, "rt", encoding='utf-8') as f:
        for line in f:
            if line[:4] == '====': break
            stHead += line
    if stHead: stHead += "% 세부 채점 내용은 첨부를 참고하기 바랍니다.\n% 채점 결과 문의 (mail to: ypjun@yonsei.ac.kr)"
#    if stHead: stHead += "% 예비 채점 결과입니다. 감점 요인을 파악하여 과제 마감 기일까지 수정 제출하여 만회하기 바랍니다."
#    if stHead: stHead += "% 채점 결과입니다. 지각 제출 기한을 4월 16일 자정까지 연장합니다. 점수가 80점 미만인 경우 다시 제출하면 20% 감점 적용 재채점하여 점수를 수정 반영하겠습니다."
    return stHead
    
stDir = os.getcwd()    
# 학생 메일주소 수집
attendee_file = './roll-addrs.xls'
gdf = pd.read_excel(attendee_file, dtype=str)

# 채점결과 폴더 선택
HomeDir = SelectDir()
class_name = os.path.basename(HomeDir)
fdBegin = class_name.find('(')
fdEnd = class_name.find(')')
if fdBegin != -1 and fdEnd != -1:
    class_name = class_name[:fdBegin].strip()

ScrDir = HomeDir + '/채점결과'
os.chdir(HomeDir)

# Sendmail 정보 생성
smTitle = class_name + " 채점 결과"

os.chdir(ScrDir)
stFolders = os.listdir(ScrDir)
bTest = False
bAll = True
rMode = input("Running mode (a: 전체 전송, i: 개별 전송): ")
if rMode == 'i':
    bAll = False

for  sfile in stFolders:
    # sfile 구조: Scoring_stName_stId.txt
    _, stName, stId = sfile.split('_')
    stId = stId.split('.')[0]
    stEmail = gdf[gdf.학번 == stId].이메일주소.values[0]
    if not stEmail:
        print("No email for {}({}) - to the next".format(stName, stId))
        continue
    smHead = getHead(sfile)
    print("==> Sendmail\n수신: {}\n제목: {}\n{}\n첨부: {}".format(stEmail, smTitle, smHead, sfile))
    if not bTest:
        print("Sending mail...")
        bSent = sendMail(stEmail, smTitle, smHead, sfile)
        if bSent == 0:
            print("SUCCESS to {}({})".format(stEmail, sfile))
        else:
            print("FAIL to {}({}) / Err code({})".format(stEmail, sfile, bSent))
    if not bAll:
        if(input("Press q to finidh...") == 'q'): break
    print('\n\n')
print("Completed...")
os.chdir(stDir)    

Running mode (a: 전체 전송, i: 개별 전송): 
==> Sendmail
수신: alsehd6382@naver.com
제목: 데이터프로그래밍 기말고사 채점 결과
데이터프로그래밍 기말고사
채점 요약 보고서 (이름: 강민송 / 학번: 2021240526)
************************************************************
1번(14.8/20.0점, 응시자 평균 18.2)  / 출력형식 불일치 (1.9점 감점) / 정답도출 실패 (3.3점 감점)
2번(17.5/20.0점, 응시자 평균 15.2)  / 출력형식 불일치 (0.9점 감점) / 정답도출 실패 (1.7점 감점)
4번(20.0/20.0점, 응시자 평균 19.5) 
5번(0.6/20.0점, 응시자 평균 4.7)  / 출력형식 불일치 (9.3점 감점) / 정답도출 실패 (10.0점 감점) / 실행오류 (10.0% 추가 감점)
총점: 53점 (응시자 37명 중 27등, 응시자 평균: 64점)

% 세부 채점 내용은 첨부를 참고하기 바랍니다.
% 채점 결과 문의 (mail to: ypjun@yonsei.ac.kr)
첨부: Scoring_강민송_2021240526.txt
Sending mail...
SUCCESS to alsehd6382@naver.com(Scoring_강민송_2021240526.txt)



==> Sendmail
수신: yelltree@naver.com
제목: 데이터프로그래밍 기말고사 채점 결과
데이터프로그래밍 기말고사
채점 요약 보고서 (이름: 김여린 / 학번: 2021240474)
************************************************************
1번(20.0/20.0점, 응시자 평균 18.2) 
2번(20.0/20.0점, 응시자 평균 15.2) 
3번(20.0/20.0점, 응시자 평균 6.2) 
4번(20.0/20.0점, 응시자 평균 19.5) 
5번(20.0/20.0점, 응시자 평균 4.7) 


SUCCESS to ahgud1216@naver.com(Scoring_박효경_2021240187.txt)



==> Sendmail
수신: backg5730@yonsei.ac.kr
제목: 데이터프로그래밍 기말고사 채점 결과
데이터프로그래밍 기말고사
채점 요약 보고서 (이름: 배경환 / 학번: 2021240208)
************************************************************
1번(20.0/20.0점, 응시자 평균 18.2) 
2번(20.0/20.0점, 응시자 평균 15.2) 
3번(20.0/20.0점, 응시자 평균 6.2) 
4번(20.0/20.0점, 응시자 평균 19.5) 
5번(19.3/20.0점, 응시자 평균 4.7)  / 실행오류 (3.3% 추가 감점)
총점: 99점 (응시자 37명 중 2등, 응시자 평균: 64점)

% 세부 채점 내용은 첨부를 참고하기 바랍니다.
% 채점 결과 문의 (mail to: ypjun@yonsei.ac.kr)
첨부: Scoring_배경환_2021240208.txt
Sending mail...
SUCCESS to backg5730@yonsei.ac.kr(Scoring_배경환_2021240208.txt)



==> Sendmail
수신: seogyobin@yonsei.ac.kr
제목: 데이터프로그래밍 기말고사 채점 결과
데이터프로그래밍 기말고사
채점 요약 보고서 (이름: 서교빈 / 학번: 2021240673)
************************************************************
1번(19.0/20.0점, 응시자 평균 18.2)  / 출력형식 불일치 (1.0점 감점)
2번(15.7/20.0점, 응시자 평균 15.2)  / 출력형식 불일치 (1.7점 감점) / 정답도출 실패 (2.5점 감점) / 실행오류 (2.5% 추가 감점)
3번(2.3/20.0점, 응시자 평균 6.2)  / 출력형식 불일치 (7.4점 감점) / 정답도출 실패 (10.0점 감

SUCCESS to limdy2021240209@yonsei.ac.kr(Scoring_임도연_2021240209.txt)



==> Sendmail
수신: limej4339@yonsei.ac.kr
제목: 데이터프로그래밍 기말고사 채점 결과
데이터프로그래밍 기말고사
채점 요약 보고서 (이름: 임은지 / 학번: 2021240090)
************************************************************
1번(19.3/20.0점, 응시자 평균 18.2)  / 출력형식 불일치 (0.7점 감점)
2번(2.9/20.0점, 응시자 평균 15.2)  / 출력형식 불일치 (6.8점 감점) / 정답도출 실패 (10.0점 감점) / 실행오류 (10.0% 추가 감점)
4번(20.0/20.0점, 응시자 평균 19.5) 
5번(0.7/20.0점, 응시자 평균 4.7)  / 출력형식 불일치 (9.3점 감점) / 정답도출 실패 (10.0점 감점)
총점: 43점 (응시자 37명 중 36등, 응시자 평균: 64점)

% 세부 채점 내용은 첨부를 참고하기 바랍니다.
% 채점 결과 문의 (mail to: ypjun@yonsei.ac.kr)
첨부: Scoring_임은지_2021240090.txt
Sending mail...
SUCCESS to limej4339@yonsei.ac.kr(Scoring_임은지_2021240090.txt)



==> Sendmail
수신: nalimjeong8@gmail.com
제목: 데이터프로그래밍 기말고사 채점 결과
데이터프로그래밍 기말고사
채점 요약 보고서 (이름: 정나림 / 학번: 2021242063)
************************************************************
1번(10.3/20.0점, 응시자 평균 18.2)  / 출력형식 불일치 (4.6점 감점) / 정답도출 실패 (5.0점 감점) / 실행오류 (5.0% 추가 감점)
2번(18.7/20.0점, 응시자 평균 15.2)  / 

In [16]:
a = os.getcwd()

In [34]:
os.chdir(stDir)   

In [31]:
stDir = 'C:\\Users\\YPJ\\Google 드라이브(ypjun7@gmail.com)\\JupyterWork\\GradeAnal\\2022-01(성적)\\파이썬채점'